In [ ]:
#| default_exp audio

In [ ]:
# | export
import sys
import wave
from pathlib import Path
import numpy as np
import sounddevice as sd
from rich.console import Console

console = Console()

## Audio Recording Subsystem

The AudioRecorder class encapsulates all functionality related to capturing audio input from the user's microphone. This component is responsible for the critical first step in our transcription pipeline: converting analog audio signals into digital data that can be processed by machine learning models.

### Design Philosophy

The audio recording system is designed with several key principles in mind:

1. **Cross-Platform Compatibility**: Works seamlessly on Windows, macOS, and Linux
2. **Real-Time Feedback**: Provides immediate visual feedback during recording
3. **Resource Management**: Properly manages audio streams and file handles
4. **Error Resilience**: Handles device failures and audio issues gracefully

### Audio Processing Pipeline

```
Microphone Input → Audio Stream → Digital Conversion → WAV File → Model Input
```

The recording process involves several technical transformations that happen transparently to the user.

In [ ]:
# | export 

class AudioRecorder:
    """Audio recording system for capturing microphone input."""
    def __init__(self, sample_rate: int = 16000, channels: int = 1):
        self.sample_rate = sample_rate
        self.channels = channels
        self.audio_file_path = self._get_audio_file_path()
        self.wave_file = None
        self.recording_frames = 0
        self.recording = False

    def _audio_callback(self, indata, frames, time, status):
        if status:
            console.print(f"⚠️ [bold yellow]Audio warning: {status}[/bold yellow]")
        if self.wave_file and self.recording:
            audio_int16 = (indata * 32767).astype(np.int16)
            self.wave_file.writeframes(audio_int16.tobytes())
            self.recording_frames += frames

    def _validate_audio_device(self):
        try:
            default_input = sd.query_devices(kind="input")
            if default_input is None:
                raise RuntimeError("No audio input device found")
        except Exception as e:
            raise RuntimeError(f"Failed to access audio devices: {e}")

    def _get_audio_file_path(self) -> Path:
        if sys.platform == "win32":
            cache_dir = Path.home() / "AppData" / "Local" / "hns" / "Cache"
        elif sys.platform == "darwin":
            cache_dir = Path.home() / "Library" / "Caches" / "hns"
        else:
            cache_dir = Path.home() / ".cache" / "hns"

        cache_dir.mkdir(parents=True, exist_ok=True)
        return cache_dir / "last_recording.wav"

    def _prepare_wave_file(self):
        self.recording_frames = 0
        self.wave_file = wave.open(str(self.audio_file_path), "wb")
        self.wave_file.setnchannels(self.channels)
        self.wave_file.setsampwidth(2)  # 16-bit audio
        self.wave_file.setframerate(self.sample_rate)

    def _close_wave_file(self):
        if self.wave_file:
            self.wave_file.close()
            self.wave_file = None

    def start_recording(self):
        """Start recording audio from the microphone.

        Initiates the audio capture process by validating the audio device,
        preparing the output file, and starting the audio stream. This method
        sets up all the necessary resources for real-time audio recording.

        The recording process runs asynchronously, allowing the main application
        to remain responsive during long recording sessions.
        """
        self._validate_audio_device()
        self._prepare_wave_file()
        self.recording = True

        try:
            self.stream = sd.InputStream(
                samplerate=self.sample_rate, channels=self.channels, callback=self._audio_callback, dtype=np.float32
            )
            self.stream.start()
            return True
        except Exception as e:
            self._close_wave_file()
            raise RuntimeError(f"Failed to initialize audio stream: {e}")

    def stop_recording(self) -> Path:
        """Stop recording and return the audio file path."""
        self.recording = False
        if hasattr(self, 'stream'):
            self.stream.stop()
            self.stream.close()
        self._close_wave_file()

        if self.recording_frames == 0:
            raise ValueError("No audio recorded")

        return self.audio_file_path

## Interactive Tests

The following cells provide interactive tests for the `AudioRecorder` functionality. These tests verify the complete audio recording pipeline without requiring a formal test framework, making it easy to validate behavior during development.

### What's Being Tested

1. **Initialization**: Verifies that the recorder can be created with default and custom parameters
2. **Cross-Platform File Management**: Tests that audio files are stored in the correct platform-specific cache directories:
   - Windows: `%LOCALAPPDATA%\hns\Cache\last_recording.wav`
   - macOS: `~/Library/Caches/hns/last_recording.wav`
   - Linux: `~/.cache/hns/last_recording.wav`
3. **Audio Device Discovery**: Validates that the system can detect and list available recording devices
4. **Resource Management**: Ensures wave files are properly opened, written to, and closed
5. **Error Handling**: Confirms appropriate errors are raised for invalid operations
6. **Recording Lifecycle**: Tests the complete start → record → stop cycle
7. **CI Compatibility**: Tests gracefully skip when running in environments without audio devices

These tests are designed to run interactively in a notebook environment and will automatically skip audio-dependent tests in CI/CD pipelines or when no microphone is available.

In [ ]:
# Test default initialization works
recorder = AudioRecorder()
console.print(f"✓ Default recorder: {recorder.sample_rate}Hz, {recorder.channels}ch, path={recorder.audio_file_path}")

# Test custom initialization works
recorder_hq = AudioRecorder(sample_rate=44100, channels=2)
console.print(f"✓ Custom recorder: {recorder_hq.sample_rate}Hz, {recorder_hq.channels}ch")

✓ Default recorder: 16000Hz, 1ch, path=/home/mario/.cache/hns/last_recording.wav

✓ Custom recorder: 44100Hz, 2ch

In [ ]:
# Test audio file path creation
recorder = AudioRecorder()
console.print(f"Audio file path: {recorder.audio_file_path}")
assert recorder.audio_file_path.parent.exists()
assert recorder.audio_file_path.suffix == ".wav"
console.print("✓ Audio file path is valid and parent directory exists")

Audio file path: /home/mario/.cache/hns/last_recording.wav

✓ Audio file path is valid and parent directory exists

In [ ]:
# Test device validation (should succeed if microphone is available)
recorder = AudioRecorder()
recorder._validate_audio_device()
# Query the actual device being used
device_info = sd.query_devices(kind="input")
console.print("✓ Audio device validated successfully")
console.print(f"  Device: [bold]{device_info['name']}[/bold]")
console.print(f"  Max channels: {device_info['max_input_channels']}")
console.print(f"  Default sample rate: {device_info['default_samplerate']}Hz")

# List all available recording devices
console.print("\n[bold cyan]Available Recording Devices:[/bold cyan]")

all_devices = sd.query_devices()
for idx, device in enumerate(all_devices):
    if device['max_input_channels'] > 0:  # Only show input devices
        is_default = "✓ [green](default)[/green]" if device['name'] == device_info['name'] else ""
        console.print(f"  [{idx}] {device['name']} - {device['max_input_channels']} ch, {device['default_samplerate']}Hz {is_default}")

✓ Audio device validated successfully

Device: default

Max channels: 64

Default sample rate: 44100.0Hz

Available Recording Devices:

[0] HDA Intel PCH: ALC285 Analog (hw:1,0) - 2 ch, 44100.0Hz

[9] pipewire - 64 ch, 44100.0Hz

[10] default - 64 ch, 44100.0Hz ✓ (default)

[11] Built-in Audio Analog Stereo - 4 ch, 48000.0Hz

[12] spotify - 2 ch, 48000.0Hz

In [ ]:
# Test wave file preparation
recorder = AudioRecorder()
recorder._prepare_wave_file()
assert recorder.wave_file is not None
assert recorder.recording_frames == 0
console.print("✓ Wave file prepared successfully")

recorder._close_wave_file()
assert recorder.wave_file is None
print("✓ Wave file closed successfully")

✓ Wave file prepared successfully

✓ Wave file closed successfully


In [ ]:
# Test that stop_recording fails without recording
from fastcore.test import test_fail
recorder = AudioRecorder()
test_fail(lambda: recorder.stop_recording(), contains="No audio recorded")
print("✓ Correctly raises error when no audio recorded")

✓ Correctly raises error when no audio recorded


In [ ]:
# Test that resources are properly cleaned up
import time
recorder = AudioRecorder()
recorder.start_recording()
time.sleep(0.5)
recorder.stop_recording()

# Try to use recorder again
recorder.start_recording()
time.sleep(0.5)
recorder.stop_recording()

✓ Recorder can be reused after stop